### Manual fitting and model coefficients

#### Objective:
   - understand the effect of changing one model coefficient at a time.

#### How it works:
   - the figure shows synthetic data points based on a polynomial equation with an added random error (noise) in the range of -0.1 to 0.1. Change the values of the model parameters $\beta_0$, $\beta_1$, $\beta_2$, $\beta_3$ and $\beta_4$ one at a time, and note the changes in the plot and values of $SSE$, $R^2$ and $DW$. 

#### Discussion:
   - Do you think it is possible to fit a model to such data manually? Initial values have been given to help you with the process.

In [7]:
import numpy as np
import bqplot as bqp
import ipywidgets as widgets
from IPython.display import clear_output, display, Latex

In [9]:
# synthetic input data
np.random.seed(0)
x = np.linspace(-1,1,30)
# z = x - 2 * (x ** 2) + 0.5 * (x ** 3) + 0.2 * (x ** 4) + np.random.normal(-0.1, 0.1, 30)
e = np.random.normal(-0.1, 0.1, 30)
z = x - 2 * (x ** 2) + 0.5 * (x ** 3) + e

# setting the plot figure objects
sc_x = bqp.LinearScale()
sc_y = bqp.LinearScale()
scat_MF = bqp.Scatter(x=x, y=z, colors=['orange'], default_size=10, scales = {'x':sc_x, 'y': sc_y})
lines_MF = bqp.Lines(x=x, y=[], colors=['darkblue'], scales = {'x':sc_x, 'y': sc_y})
ax_x = bqp.Axis(scale=sc_x, label='x')
ax_y = bqp.Axis(scale=sc_y, orientation='vertical', label='y')
figMF = bqp.Figure(marks=[scat_MF, lines_MF], axes=[ax_x, ax_y], title='Manual fitting', animation_duration=1000, preserve_aspect=True)
figMF.layout.width = '800px'
figMF.layout.length = '700px'

SSEtext = ''
SSELabel = widgets.Label(value = r'\(\color{red} {' + SSEtext  + '}\)')

Rsqrtext = ''
RsqrLabel = widgets.Label(value = r'\(\color{red} {' + Rsqrtext  + '}\)')

DWtext = 'bang'
DWLabel = widgets.Label(value = r'\(\color{red} {' + DWtext  + '}\)')

b4_var = widgets.FloatText(value=0.5, step=0.1, description=r'\(\beta_4\)')
b3_var = widgets.FloatText(value=0.5, step=0.1, description=r'\(\beta_3\)')
b2_var = widgets.FloatText(value=-1.5977, step=0.1, description=r'\(\beta_2\)')
b1_var = widgets.FloatText(value=0.6048, step=0.1, description=r'\(\beta_1\)')
b0_var = widgets.FloatText(value=-0.0922, step=0.1, description=r'\(\beta_0\)')

# function to update plot based on input model parameters
def update_plot(*args):
    b4 = b4_var.value
    b3 = b3_var.value
    b2 = b2_var.value
    b1 = b1_var.value
    b0 = b0_var.value
    
    lines_MF.y = b4*x**4 + b3*x**3 + b2*x**2 + b1*x + b0
    SSE = np.sum((z-lines_MF.y)**2)
    SST = np.sum((z - np.mean(z))**2)
    Rsqr = 1 - SSE/SST
    err = z - lines_MF.y
    DW = (np.sum((np.diff(err))**2))/sum(err**2)
    
    SSEtext = 'SSE={:.3f}'.format(SSE)
    SSELabel.value = r'\(\color{black} {' + SSEtext  + '}\)'

    Rsqrtext = 'R^2={:.3f}'.format(Rsqr)
    RsqrLabel.value = r'\(\color{black} {' + Rsqrtext  + '}\)'

    DWtext = 'DW={:.3f}'.format(DW)
    DWLabel.value = r'\(\color{black} {' + DWtext  + '}\)'
    
    return DWtext

b4_var.observe(update_plot, names = 'value')
b3_var.observe(update_plot, names = 'value')
b2_var.observe(update_plot, names = 'value')
b1_var.observe(update_plot, names = 'value')
b0_var.observe(update_plot, names = 'value')

# create a 9x4 grid layout
grid = widgets.GridspecLayout(9, 4)

# fill it in with widgets
grid[:, 0:2] = figMF
grid[0, 3] = b4_var
grid[1, 3] = b3_var
grid[2, 3] = b2_var
grid[3, 3] = b1_var
grid[4, 3] = b0_var
grid[6, 3] = SSELabel
grid[7, 3] = RsqrLabel
grid[8, 3] = DWLabel

update_plot(None)
display(grid)

GridspecLayout(children=(Figure(animation_duration=1000, axes=[Axis(label='x', scale=LinearScale()), Axis(labe…

### OLS, Polynomial degrees and data error range
#### Objective:  
   - understand the effect of changing the degree of a polynomial model on the OLS evaluation values.
   - understand the effect of changing the error range in the data that need fitting

#### How it works:
   - **Degree:** refers to the degree of the polynomial model (range: 0-28, step: 1).
   - **Error range:** specifies the synthetic random error range added to the data points, and Error range of 0.2, means the error for each data point can be anything from -0.2 to 0.2 (range: 0-1, step: 0.1).
   - **Polynomials**: the same data points are used as in the previous task. Change the polynomial degree and note the changes in model parameters (coef) and other values shown in the OLS Regression Results table.
   - **Residuals**: a plot for the analysis of residuals.
   
#### Discussion
   - Based on your understanding from the session, what is the best model equation that can fit this sample of data at error range 0.1 (same as previous task)? Compare the OLS parameters results to the values you tried in the last task.
   - Does changing the error range has an effect on the model to be chosen? What does this tell you?

In [3]:
from sklearn.linear_model import LinearRegression

In [4]:
from sklearn.preprocessing import PolynomialFeatures

In [5]:
import statsmodels.api as sm

In [8]:
# same synthetic data used in exercise 1
np.random.seed(0)
x = np.linspace(-1,1,30)
e = np.random.normal(-0.1, 0.1, 30)
z = x - 2 * (x ** 2) + 0.5 * (x ** 3) + e

# setting the plot figure objects
sc_x = bqp.LinearScale()
sc_y = bqp.LinearScale()
scat_PD = bqp.Scatter(x=x, y=[], colors=['orange'], default_size=10, scales = {'x':sc_x, 'y': sc_y})
lines_PD = bqp.Lines(x=x, y=[], colors=['darkblue'], scales = {'x':sc_x, 'y': sc_y})
ax_x = bqp.Axis(scale=sc_x, label='x')
ax_y = bqp.Axis(scale=sc_y, orientation='vertical', label='y')
figPD = bqp.Figure(marks=[scat_PD, lines_PD], axes=[ax_x, ax_y], title='Polynomials', animation_duration=1000, preserve_aspect=True)
figPD.layout.width = '600px'
figPD.layout.height = '350px'

ax_xRsd = bqp.Axis(scale=sc_x, label='x')
ax_yRsd = bqp.Axis(scale=sc_y, orientation='vertical', label='y')
scat_Rsd = bqp.Scatter(x=x, y=[], colors=['orange'], default_size=10, scales = {'x':sc_x, 'y': sc_y})
figRsd = bqp.Figure(marks=[scat_Rsd], axes=[ax_xRsd, ax_yRsd], title='Residuals', animation_duration=1000, preserve_aspect=False)
figRsd.layout.width = '600px'
figRsd.layout.height = '250px'

errRange = widgets.FloatText(value=0.1, min=0, max=1, step=0.1, description='Error range')
Degree = widgets.IntText(value=0, min=0, max=28, description='Degree')
out = widgets.Output(layout={'border': '1px solid black'}, clear=True)

# define function for plot update when changing the degree widget
def update_plot(*args):
    deg = Degree.value
    err = errRange.value
    if deg >= 0:
        # use sklearn functions to fit the data given different model degrees
        np.random.seed(0)
        e = np.random.normal(-err, err, 30)
        z = x - 2 * (x ** 2) + 0.5 * (x ** 3) + e
        x1 = x[:, np.newaxis]
        y1 = z[:, np.newaxis]
        polynomial_features= PolynomialFeatures(degree=deg)
        x_poly = polynomial_features.fit_transform(x1)
        model = LinearRegression()
        model.fit(x_poly, y1)
        y_poly_pred = model.predict(x_poly)
        X = x_poly
    
        # plot the updated model and data
        lines_PD.y = y_poly_pred
        scat_PD.y = z
        scat_Rsd.y = z - lines_PD.y
        # print the updated OLS regression results
        model = sm.OLS(z, X)
        results = model.fit()
        Summary = results.summary()
        # model evaluation calculations
        n = len(z)
        k = deg + 1
        SSE = np.sum((z-lines_PD.y)**2)
        SST = np.sum((z - np.mean(z))**2)
        SSR = SST - SSE
        Rsqr = 1 - SSE/SST
        errDW = z - lines_PD.y
        DW = (np.sum((np.diff(errDW))**2))/sum(errDW**2)
        R2_corrected = 1-(1-Rsqr)*((n-1)/(n-k))
        RMSE = np.sqrt(np.sum((z-lines_PD.y)**2)/(n-k))
        MBE = np.sum(z-lines_PD.y)/(n-k)
        if k > 1:
            f = (SSR/SSE)*((n-k)/(k-1))
        else:
            f = 'nan'
            
        with out:
            out.clear_output()
            display(Latex('=============== MODEL COEFFICIENTS ================'))
            for attributeIndex in range (0, deg+1):
                display(Latex(f'b{attributeIndex} = {results.params[attributeIndex].round(4)}')) 
            display(Latex('=============== COEFFICIENTS P-VALUES ==============='))
            for attributeIndex in range (0, deg+1):
                display(Latex(f'p-value for b{attributeIndex} = {results.pvalues[attributeIndex].round(4)}'))
            display(Latex('=============== MODEL EVALUATION ================'))
            display(Latex(f'RMSE = {RMSE.round(4)}'))
            display(Latex(f'MBE = {MBE.round(4)}'))
            display(Latex(f'R$^2$ = {Rsqr.round(4)}'))
            display(Latex(f'R$^2$ adjusted = {R2_corrected.round(4)}'))
            display(Latex(f'Durbin Watson = {DW.round(4)}'))
            if f == 'nan':
                display(Latex(f'F-statistic = nan'))
            else:
                display(Latex(f'F-statistic = {f.round(4)}'))
    else:
        with out:
            out.clear_output()
            print("Polynomial Degree has to be larger than 0")   

    
# observe the widgets and call function update_plot
Degree.observe(update_plot, names='value')
errRange.observe(update_plot, names='value')

# create a 1x5 grid layout
grid = widgets.GridspecLayout(1, 5)

upperbox = widgets.HBox([Degree, errRange])
leftbox = widgets.VBox([upperbox, figPD, figRsd])
# box = widgets.HBox([leftbox, out])
# fill it in with widgets
grid[:, 0:3] = leftbox
grid[:, 3:5] = out

update_plot(None)
display(grid)
# display(box)

GridspecLayout(children=(VBox(children=(HBox(children=(IntText(value=0, description='Degree'), FloatText(value…